In [1]:
from nilearn.plotting import plot_img_on_surf
from nilearn import surface, datasets, image
import nibabel as nib
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from nilearn.image import resample_to_img
import os, glob

In [2]:
# ─── adjust if you changed any names  ─────────────────────────────
stim_label = "notthefallintact"

base = Path("/Volumes/Passport/fmriprep/derivatives")
perm_dir = base / "RSA_stats" / stim_label / "perm_test_neural_shift"
model_key   = "trait_9"                    # Choose model key (mental_8, personality_5, all_13, trait_9)
smoothing_setting     = "_no_smoothing"              # set smoothing to _no_smoothing or ""
template = datasets.load_mni152_template()    
out_dir     = perm_dir / "figs";  out_dir.mkdir(exist_ok=True)

p_img_path  = perm_dir / f"{stim_label}_perm_pmap_{model_key}_k1{smoothing_setting}.nii.gz"



In [ ]:
# 1) load volume & fetch fsaverage5 (≈10 k vertices) ─────────────
p_img_unthresholded  = nib.load(p_img_path)
fsavg  = datasets.fetch_surf_fsaverage(mesh="fsaverage5")

# helper ─ convert a volume → surface texture for both hems
def vol_to_surf_pair(img):
    tex_l = surface.vol_to_surf(img, fsavg.pial_left,  radius=3.)
    tex_r = surface.vol_to_surf(img, fsavg.pial_right, radius=3.)
    return tex_l, tex_r


In [4]:
# --- load & resample Schaefer atlas to match p_img space ----------
schaefer   = datasets.fetch_atlas_schaefer_2018(
                 n_rois=200, yeo_networks=17,
                 resolution_mm=2
             )
atlas_img  = nib.load(schaefer["maps"])
atlas_res  = image.resample_to_img(
                 atlas_img, p_img, interpolation="nearest"
             )
atlas_data = atlas_res.get_fdata().astype(int)

In [5]:
import numpy as np
import matplotlib.pyplot as plt

def fix_colorbar_ticks(fig_or_tuple, lo, hi, n=6, fmt="{:.2f}"):
    """
    Force evenly spaced ticks on the colour-bar of a figure produced by
    plot_img_on_surf, regardless of nilearn version.

    Parameters
    ----------
    fig_or_tuple : Figure | (Figure, ...)
        Whatever plot_img_on_surf returned (figure alone or tuple).
    lo, hi       : float
        Lower and upper limits used for vmin / vmax.
    n            : int
        Number of ticks (default 6).
    fmt          : str
        Format string for tick labels (e.g., "{:.02f}").
    """
    # ----------------------------------------------------------
    # 1) get the Figure object
    # ----------------------------------------------------------
    if isinstance(fig_or_tuple, tuple):
        fig = fig_or_tuple[0]     # first item is always the Figure
    else:
        fig = fig_or_tuple

    # ----------------------------------------------------------
    # 2) colour-bar axis is always the last one added
    # ----------------------------------------------------------
    cbar_ax = fig.axes[-1]

    # ----------------------------------------------------------
    # 3) apply ticks / labels
    # ----------------------------------------------------------
    ticks = np.linspace(lo, hi, n)
    labels = [fmt.format(t) for t in ticks]

    if cbar_ax.xaxis.get_visible():          # horizontal colour-bar
        cbar_ax.set_xticks(ticks)
        cbar_ax.set_xticklabels(labels)
    else:                                    # vertical colour-bar
        cbar_ax.set_yticks(ticks)
        cbar_ax.set_yticklabels(labels)

In [ ]:
# -----------------------------------------------------------------
# SECOND MAP  ▸  1-p “significance” map   (retain 1-p ≥ 0.95)
# -----------------------------------------------------------------
inv_unthresholded_label    = f"{stim_label}_inv_pmap_{model_key}_neural_shift_unthresholded"


# --- get raw p-values and FIX zeros ---------------------------
p_dat = p_img_unthresholded.get_fdata()
p_dat[p_dat == 0] = 1.0           # ← treat background/empty voxels as p = 1

# --- compute 1-p & mask -----------------------------------------
inv_p_data   = 1.0 - p_dat

# --- force atlas==0 voxels → nan in the 1-p map --------------------
inv_p_data[atlas_data == 0] = np.nan


inv_img      = nib.Nifti1Image(inv_p_data, affine=p_img_unthresholded.affine,
                               header=p_img_unthresholded.header)

corrected_nifti_path = out_dir / f"{inv_unthresholded_label}.nii.gz"
nib.save(inv_img, corrected_nifti_path)
print("✅  Corrected 1–p NIfTI written →", corrected_nifti_path)


# --- plot on surface --------------------------------------------
fig2 = plot_img_on_surf(
    inv_img,
    surf_mesh='fsaverage5',
    views=['lateral', 'medial'],
    hemispheres=['left', 'right'],
    title=f"{stim_label} {inv_unthresholded_label} (1-p, k=1 {smoothing_setting})",
    colorbar=True,
    inflate=True,
    cmap='autumn',        # dark→bright as significance ↑
    vmin= 0,   
    vmax=1.0,
    threshold=None,
    bg_on_data=True
)
fix_colorbar_ticks(fig2, lo = 0, hi = 1, n=2, fmt="{:.02f}")   
# --- save --------------------------------------------------------
out_path2 = out_dir / f"{inv_unthresholded_label}.png"
plt.savefig(out_path2, dpi=300, bbox_inches='tight')
plt.close('all')

print("✅  Figure written →", out_path2)

✅  Corrected 1–p NIfTI written → /Volumes/Passport/fmriprep/derivatives/RSA_stats/notthefallintact/perm_test_neural_shift/figs/notthefallintact_inv_pmap_trait_9_neural_shift_unthresholded.nii.gz


/opt/anaconda3/lib/python3.12/site-packages/nilearn/surface/surface.py:464: RuntimeWarning: Mean of empty slice
  texture = np.nanmean(all_samples, axis=2)
/opt/anaconda3/lib/python3.12/site-packages/nilearn/surface/surface.py:464: RuntimeWarning: Mean of empty slice
  texture = np.nanmean(all_samples, axis=2)


✅  Figure written → /Volumes/Passport/fmriprep/derivatives/RSA_stats/notthefallintact/perm_test_neural_shift/figs/notthefallintact_inv_pmap_trait_9_neural_shift_unthresholded.png
